
# filesystem

a notebook full of filesystem utilities




## organize downloads




### organize downloads?

organize downloads folder using AI?


In [ ]:

//let promptModel = await selectModel(process.env.DEFAULT_MODEL || 'Default')







categorize home directory?


In [ ]:
const path = require('path')
const fs = require('fs')
const selectModel = importer.import('select llm')

const PROJECT_PATH = path.join(path.dirname(__dirname), '.categories.json')
const PROFILE_HOME = path.resolve(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE)

let existingCategories = []
let mtime = ''
if (fs.existsSync(PROJECT_PATH)) {
  existingCategories = JSON.parse(fs.readFileSync(PROJECT_PATH))
  mtime = fs.statSync(PROJECT_PATH).mtime.toISOString()
}

async function categorizeHome() {
  let promptModel = await selectModel(process.env.DEFAULT_MODEL || 'Gemma')

  let basepath = PROFILE_HOME

  let projects = fs.readdirSync(basepath)
    .filter(dir => dir[0] != '.'
      && fs.statSync(path.join(basepath, dir)).isDirectory()
      && fs.existsSync(path.join(basepath, dir, '.git'))
      && fs.statSync(path.join(basepath, dir, '.git')).isDirectory()
      && !dir.includes('porn'))

  let q1 = 'Here\'s a list of existing projects and modified times:\n'
    + projects.map(dir => dir + ' - ' + fs.statSync(path.join(basepath, dir)).mtime.toISOString()).join('\n')
    + (existingCategories.length == 0 ? '' : ('\nHere is the list of existing categories modified on '
      + mtime + ' :\n' + existingCategories.join('\n')))
    + '\nPlease make an awesome list of categories, keep the category name simple, like one or two words.'
    + '\nAdd any new categories that seem necessary to accomodate any new projects.'
    + '\nOnly respond with the flat new line delimited list of categories and nothing else.'
  console.log('User:', q1)
  let a1 = await promptModel(q1)
  console.log('AI:', a1)

  let categories = a1.trim().split(/\s*\n\s*|,\s*|\s*- |\s*\*+\s*/gi)
  let uniqueCategories = existingCategories.concat(categories).filter((a, i, arr) => arr.indexOf(a) === i)

  fs.writeFileSync(PROJECT_PATH, JSON.stringify(uniqueCategories, null, 4))
  // TODO: cache categories
  return categories

}

module.exports = categorizeHome




categories downloads?


In [ ]:
const path = require('path')
const fs = require('fs')
const selectModel = importer.import('select llm')

const PROJECT_PATH = path.join(path.dirname(__dirname), '.categories.json')
const PROJECT_HOME = path.join(path.resolve(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE), 'Downloads')

let existingCategories = []
let mtime = ''
if (fs.existsSync(PROJECT_PATH)) {
  existingCategories = JSON.parse(fs.readFileSync(PROJECT_PATH))
  mtime = fs.statSync(PROJECT_PATH).mtime.toISOString()
}

async function promptBatch(batchDownloads, basepath) {
  let promptModel = await selectModel(process.env.DEFAULT_MODEL || 'Gemma')

  let q1 = 'Here\'s a partial list of existing files and modified times:\n'
    + batchDownloads
    + (existingCategories.length == 0 ? '' : ('\nHere is the list of existing categories modified on '
      + mtime + ' :\n' + existingCategories.join('\n')))
    + '\nPlease make a short list of categories, keep the category name simple, like one or two words.'
    + '\nOnly list the missing category for this partial list of files if necessary.'
    + '\nOnly respond with the flat new line delimited list of categories and nothing else.'
  console.log('User:', q1)
  let a1 = await promptModel(q1)
  console.log('AI:', a1)

  let categories = a1.trim().split(/\s*\n\s*|,\s*|\s*- |\s*\*+\s*/gi)
  existingCategories = existingCategories.concat(categories).filter((a, i, arr) => arr.indexOf(a) === i)
  fs.writeFileSync(PROJECT_PATH, JSON.stringify(existingCategories, null, 4))

  return categories
}

async function categorizeDownloads() {
  let basepath = PROJECT_HOME

  let downloads = fs.readdirSync(basepath).sort()
    .filter(dir => {
      let now = Date.now()
      let stats = fs.statSync(path.join(basepath, dir))
      // filetime + 30 days > is still in the future
      let isRecent = stats.mtime.getTime() + 30 * 24 * 60 * 60 * 1000 > now
      if(!isRecent) {
        isRecent = stats.atime.getTime() + 30 * 24 * 60 * 60 * 1000 > now
      }
      if(!isRecent) {
        isRecent = stats.ctime.getTime() + 30 * 24 * 60 * 60 * 1000 > now
      }
      return !isRecent
    }) // files being used
    .filter(dir => dir[0] != '.' && !dir.includes('porn'))
    .filter(dir => !existingCategories.includes(dir))

  let batchDownloads = ''
  for (let i = 0; i < downloads.length; i++) {
    if (batchDownloads.length + downloads[i].length > 4000) {
      await promptBatch(batchDownloads, basepath)
      batchDownloads = ''
    }
    batchDownloads += (batchDownloads.length > 0 ? '\n' : '')
      + downloads[i] + ' - ' + fs.statSync(path.join(basepath, downloads[i])).mtime.toISOString()
  }
  if (batchDownloads.length > 0) {
    await promptBatch(batchDownloads, basepath)
  }

  // TODO: cache categories
  return existingCategories

}

module.exports = categorizeDownloads




download reorganize bash?


In [ ]:
const path = require('path')
const fs = require('fs')
const selectModel = importer.import('select llm')

const PROJECT_PATH = path.join(path.dirname(__dirname), '.categories.json')
const PROJECT_HOME = path.join(path.resolve(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE), 'Downloads')

let existingCategories = []
let mtime = ''
if (fs.existsSync(PROJECT_PATH)) {
  existingCategories = JSON.parse(fs.readFileSync(PROJECT_PATH))
  mtime = fs.statSync(PROJECT_PATH).mtime.toISOString()
}

async function promptBatch(batchDownloads, basepath) {
  let promptModel = await selectModel(process.env.DEFAULT_MODEL || 'Gemma')

  let q1 = 'Here\'s a partial list of existing files:\n'
    + batchDownloads.replaceAll('$', '\\$$')
    + (existingCategories.length == 0 ? '' : ('\nHere is the list of existing categories modified on '
      + mtime + ' :\n' + existingCategories.join('\n')))
    + '\nPlease make a Shell script (' + require('os').platform() + ') that lists `mv` commands to move the files into their most relevant categorized directory.'
    + '\nFor example the command for .pdf files might be `mv "file.pdf" "./PDFs/" || :`. Skip the repeat filename as files will go to their new directory.'
    + '\nOnly respond with the flat new line delimited list of bash commands and nothing else.'
  console.log('User:', q1)
  let a1 = await promptModel(q1)
  console.log('AI:', a1)

  return a1
}

// const getDateAdded = (filePath) => execSync(`mdls -name kMDItemDateAdded -raw "${filePath}"`).toString().trim();

async function reorganizeBash() {
  let basepath = PROJECT_HOME

  let downloads = fs.readdirSync(basepath)
  downloads.sort((a, b) => a.toLocaleLowerCase().localeCompare(b.toLocaleLowerCase()))
  downloads = downloads.filter(dir => {
      let now = Date.now()
      let stats = fs.statSync(path.join(basepath, dir))
      // filetime + 30 days > is still in the future
      //console.log(stats.mtime.toISOString(), stats.mtime.getTime() + 30 * 24 * 60 * 60 * 1000 , now, stats.mtime.getTime() + 30 * 24 * 60 * 60 * 1000 > now)
      let isRecent = stats.mtime.getTime() + 30 * 24 * 60 * 60 * 1000 > now
      //if(!isRecent) {
      //  isRecent = stats.atime.getTime() + 30 * 24 * 60 * 60 * 1000 > now
      //}
      //if(!isRecent) {
      //  isRecent = stats.ctime.getTime() + 30 * 24 * 60 * 60 * 1000 > now
      //}
      return !isRecent
    }) // files being used
    .filter(dir => dir[0] != '.' && !dir.includes('porn'))
    .filter(dir => !existingCategories.includes(dir))

  let batchDownloads = ''
  for (let i = 0; i < downloads.length; i++) {
    if (batchDownloads.length + downloads[i].length > 2000) {
      // only run once?
      return await promptBatch(batchDownloads, basepath)
      batchDownloads = ''
    }
    batchDownloads += (batchDownloads.length > 0 ? '\n' : '') + downloads[i]
  }
  if (batchDownloads.length > 0) {
    return await promptBatch(batchDownloads, basepath)
  }

  throw new Error('No files to reorganize') // so docker doesn't accidentally execute
}

module.exports = reorganizeBash




bash make categories?


In [ ]:
const path = require('path')
const fs = require('fs')
const selectModel = importer.import('select llm')

const PROJECT_PATH = path.join(path.dirname(__dirname), '.categories.json')

let existingCategories = []
let mtime = ''
if (fs.existsSync(PROJECT_PATH)) {
  existingCategories = JSON.parse(fs.readFileSync(PROJECT_PATH))
  mtime = fs.statSync(PROJECT_PATH).mtime.toISOString()
}

async function bashCategories() {
  let promptModel = await selectModel(process.env.DEFAULT_MODEL || 'Gemma')

  let q1 = (existingCategories.length == 0 ? '' : ('\nHere is the list of existing categories modified on '
      + mtime + ' :\n' + existingCategories.join('\n')))
    + '\nPlease make a Shell script (' + require('os').platform() + ') that lists `mkdir` commands to make category directories.'
    + '\nFor example the command for Taxes might be `if [ ! -d "Taxes" ]; then mkdir "./Taxes"; fi; || :`. Skip the repeat filename as files will go to their new directory.'
    + '\nOnly respond with the flat new line delimited list of bash commands and nothing else.'
  console.log('User:', q1)
  let a1 = await promptModel(q1)
  console.log('AI:', a1)

  return a1
}

module.exports = bashCategories



run bash on downloads?


In [ ]:
const fs = require('fs')
const path = require('path')
const {spawnSync} = require('child_process')
const {safeurl} = importer.import('domain cache tools')

const PROJECT_PATH = path.resolve(path.dirname(__dirname))
const DOWNLOAD_PATH = path.join(path.resolve(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE), 'Downloads')
const CD_DIR = '/home/' + path.basename(DOWNLOAD_PATH)

async function runBashCommands(prompt = 'download reorganize bash') {

  let bashGeneratingFunction = await importer.import(prompt)
  console.log('calling bash generator:', bashGeneratingFunction)
  if(typeof bashGeneratingFunction == 'object' && typeof bashGeneratingFunction[Object.keys(bashGeneratingFunction)[0]] == 'function') {
    bashGeneratingFunction = bashGeneratingFunction[Object.keys(bashGeneratingFunction)[0]]
  }

  let codeBlocks = await bashGeneratingFunction()

  if (!fs.existsSync(path.join(PROJECT_PATH, '.automations'))) {
    fs.mkdirSync(path.join(PROJECT_PATH, '.automations'))
  }
  const CODE_FILE = path.join('.automations', safeurl((new Date).toISOString() + '-' + prompt) + '.sh')
  fs.writeFileSync(path.join(PROJECT_PATH, CODE_FILE), 'cd ' + CD_DIR + ';\n' + codeBlocks)
  fs.chmodSync(path.join(PROJECT_PATH, CODE_FILE), 0o777)

  let baseName = path.basename(PROJECT_PATH)
  // TODO: execute new docker file on project
  let args = ['run', '-it', '--rm',
    // TODO: mount project directory
    '-v', PROJECT_PATH + ':/home/' + baseName,
    '-v', DOWNLOAD_PATH + ':' + CD_DIR,
    // TODO: change docker image
    'alpine:latest', 'sh',
    path.join('/home/', baseName, CODE_FILE)]
  console.log('docker', args.join(' '))
  await spawnSync('docker', args, {
    cwd: PROJECT_PATH,
    timeout: 3000,
    stdio: [0, 1, 2]
  })
}

module.exports = runBashCommands




macos time added in objective-c

macos time added?

FRAMEWORKS = CoreServices


In [ ]:
#include <CoreServices/CoreServices.h>
#include <time.h>

#ifdef __cplusplus
extern "C" {
#endif

// Exported function to retrieve "Date Added" as Unix timestamp.
// Returns -1 if metadata not available or file not indexed.
time_t get_date_added_unix_timestamp(const char *filePath) {
    if (!filePath) return -1;

    CFStringRef path = CFStringCreateWithCString(NULL, filePath, kCFStringEncodingUTF8);
    if (!path) return -1;

    CFURLRef fileURL = CFURLCreateWithFileSystemPath(NULL, path, kCFURLPOSIXPathStyle, false);
    CFRelease(path);
    if (!fileURL) return -1;

    MDItemRef item = MDItemCreateWithURL(NULL, fileURL);
    CFRelease(fileURL);
    if (!item) return -1;

    CFTypeRef dateAdded = MDItemCopyAttribute(item, kMDItemDateAdded);
    CFRelease(item);
    if (!dateAdded) return -1;

    time_t unixTime = -1;
    if (CFGetTypeID(dateAdded) == CFDateGetTypeID()) {
        CFDateRef date = (CFDateRef)dateAdded;
        CFAbsoluteTime absTime = CFDateGetAbsoluteTime(date);
        unixTime = (time_t)(absTime + kCFAbsoluteTimeIntervalSince1970);
    }

    CFRelease(dateAdded);
    return unixTime;
}

#ifdef __cplusplus
}
#endif
